In [1]:
#Importing Necessary Libraries.
import pandas as pd
import numpy as np
from scipy.stats import gmean
from itertools import groupby

In [2]:
#Importing Expert PairWise Matrices and combining them into a single PairWise Matrix.
PairWise_Matrices = pd.read_excel("AHP Expert Data.xlsx", sheet_name = None)
keys = list(PairWise_Matrices.keys())
print(keys)
print("\n")

i = 1
count_Experts = 0
for n in range(len(PairWise_Matrices)):
    globals()["Expert" + str(i)] = PairWise_Matrices[keys[count_Experts]]
    globals()["Expert" + str(i)] = globals()["Expert" + str(i)].set_index("CRITERIA")
    globals()["Expert" + str(i) + "_1"] = np.array(globals()["Expert" + str(i)])
    i += 1
    count_Experts += 1

print("Total no of Experts: ", count_Experts)
print("\n")

PairWise_Matrix_comb = np.ones([len(Expert1),len(Expert1)])
for i in range(1, count_Experts+1):
    PairWise_Matrix_comb = PairWise_Matrix_comb * globals()["Expert" + str(i) + "_1"]

PairWise_Matrix_comb = np.power(PairWise_Matrix_comb, 1/len(Expert1))


PairWise_Matrix_comb_1 = pd.DataFrame(PairWise_Matrix_comb, columns = Expert1.columns)
PairWise_Matrix_comb_1["CRITERIA"] = Expert1.index
PairWise_Matrix_comb_1 = PairWise_Matrix_comb_1.set_index("CRITERIA")
print(PairWise_Matrix_comb_1)
PairWise_Matrix_comb_1.to_excel("Pair Wise Matrix Combined.xlsx")

['Exp1', 'Exp2', 'Exp3', 'Exp4', 'Exp5', 'Exp6', 'Exp7', 'Exp8', 'Exp9', 'Exp10', 'Exp11', 'Exp12', 'Exp13', 'Exp14', 'Exp15']


Total no of Experts:  15


                AC        BC       LC         TC         S        PH  \
CRITERIA                                                               
AC        1.000000  1.099299  1.121274  1.121274  0.937420  0.891843   
BC        0.909671  1.000000  1.099299  1.099299  0.909671  0.878756   
LC        0.891843  0.909671  1.000000  1.066758  0.899967  0.878756   
TC        0.891843  0.909671  0.937420  1.000000  0.891843  0.878756   
S         1.066758  1.099299  1.111152  1.121274  1.000000  0.909671   
PH        1.121274  1.137973  1.137973  1.137973  1.099299  1.000000   
RA        1.099299  1.111152  1.121274  1.111152  1.066758  0.891843   
R         1.066758  1.066758  1.099299  1.084964  0.909671  0.899967   
D         1.084964  1.099299  1.121274  1.121274  0.960047  0.909671   
EC        0.884865  0.909671  0.960047  0.937420  0.

In [3]:
#Obtaining Criteria Weights.
colWise_Sum = PairWise_Matrix_comb_1.loc[:].sum(axis = 0)
PWMatrix_with_colSum = PairWise_Matrix_comb_1.append(colWise_Sum, ignore_index=True)

for i in range(0, len(PWMatrix_with_colSum.columns)):
    sum_val = PWMatrix_with_colSum.iloc[-1,i]
    for j in range(0, len(PWMatrix_with_colSum)-1):
        if sum_val > 0:
            PWMatrix_with_colSum.iloc[j,i] = PWMatrix_with_colSum.iloc[j,i] / sum_val

PWMatrix_with_colSum.drop(PWMatrix_with_colSum.tail(1).index,inplace=True)
PWMatrix_with_colSum.to_excel("Normalized Pair Wise Matrix.xlsx")
print("\n")   
n = len(PWMatrix_with_colSum)
rowWise_Sum = PWMatrix_with_colSum.loc[:].mean(axis = 1)
PWMatrix_with_colSum["Criteria Weights"] = rowWise_Sum
print(PWMatrix_with_colSum)
PWMatrix_with_colSum.to_excel("Matrix with Criteria Weights.xlsx")

Criteria_Wts = PWMatrix_with_colSum["Criteria Weights"].tolist()
Criteria_Wts_1 = pd.DataFrame(Criteria_Wts)
Criteria_Wts_1.to_excel("Weightages.xlsx")
Criteria_Wts = np.array(Criteria_Wts)



          AC        BC       LC         TC         S        PH        RA  \
0   0.057047  0.061254  0.061213  0.061126  0.057307  0.057563  0.054098   
1   0.051894  0.055721  0.060013  0.059928  0.055610  0.056718  0.053520   
2   0.050877  0.050688  0.054592  0.058154  0.055017  0.056718  0.053037   
3   0.050877  0.050688  0.051176  0.054515  0.054520  0.056718  0.053520   
4   0.060855  0.061254  0.060660  0.061126  0.061132  0.058713  0.055748   
5   0.063965  0.063409  0.062124  0.062036  0.067203  0.064544  0.066681   
6   0.062712  0.061915  0.061213  0.060574  0.065213  0.057563  0.059469   
7   0.060855  0.059441  0.060013  0.059146  0.055610  0.058087  0.054098   
8   0.061894  0.061254  0.061213  0.061126  0.058690  0.058713  0.053520   
9   0.050479  0.050688  0.052411  0.051103  0.056345  0.057112  0.053037   
10  0.054768  0.059441  0.056864  0.059928  0.063676  0.058087  0.062508   
11  0.062712  0.061915  0.061213  0.061126  0.055610  0.057563  0.055748   
12  0.0618

In [4]:
#CONSISTENCY CHECK:
PairWise_Matrix_comb_1 = np.array(PairWise_Matrix_comb_1)
Consistency_check_Matrix = PairWise_Matrix_comb_1 * Criteria_Wts
Consistency_check_Matrix = pd.DataFrame(Consistency_check_Matrix, columns = Expert1.columns)
Consistency_check_Matrix["CRITERIA"] = Expert1.index
Consistency_check_Matrix = Consistency_check_Matrix.set_index("CRITERIA")

rowSum = Consistency_check_Matrix.loc[:].sum(axis = 1)
Consistency_check_Matrix["WSV"] = rowSum
Consistency_check_Matrix.to_excel("Consistency Check Matrix.xlsx")
Consistency_check_Matrix["Criteria Weights"] = list(Criteria_Wts)

Consistency_check_Matrix['WSV / CW'] = Consistency_check_Matrix['WSV'] / Consistency_check_Matrix['Criteria Weights'] 
print(Consistency_check_Matrix)
Consistency_check_Matrix.to_excel("Consistency Check.xlsx")
print("\n")

λmax = gmean(Consistency_check_Matrix["WSV / CW"])
print("λmax: ", λmax)

                AC        BC       LC         TC         S        PH  \
CRITERIA                                                               
AC        0.057148  0.061300  0.061191  0.061089  0.057364  0.057492   
BC        0.051986  0.055763  0.059991  0.059892  0.055666  0.056649   
LC        0.050967  0.050726  0.054572  0.058119  0.055072  0.056649   
TC        0.050967  0.050726  0.051157  0.054482  0.054575  0.056649   
S         0.060963  0.061300  0.060638  0.061089  0.061193  0.058641   
PH        0.064079  0.063457  0.062102  0.061999  0.067269  0.064464   
RA        0.062823  0.061961  0.061191  0.060538  0.065278  0.057492   
R         0.060963  0.059486  0.059991  0.059111  0.055666  0.058016   
D         0.062004  0.061300  0.061191  0.061089  0.058748  0.058641   
EC        0.050568  0.050726  0.052392  0.051073  0.056401  0.057042   
P         0.054865  0.059486  0.056844  0.059892  0.063740  0.058016   
MUD       0.062823  0.061961  0.061191  0.061089  0.055666  0.05

In [5]:
#Obtaining Consistency Ratio.
Consistency_index = (λmax - len(Expert1)) / (len(Expert1) - 1)
print("CI: ", Consistency_index)

n = len(Expert1)
if n > 0 and n < 51:
    Random_Index_vals={1:0, 2:0, 3:0.58, 4:0.90, 5:1.12, 6:1.24, 7:1.32, 8:1.41, 9:1.45, 10:1.49, 11:1.51, 12:1.48, 13:1.56, 14:1.57, 
                       15:1.59, 16:1.6, 17:1.61, 18:1.61, 19:1.62, 20:1.63, 21:1.63, 22:1.64, 23:1.65, 24:1.65, 25:1.66, 26:1.66, 
                       27:1.66, 28:1.67, 29:1.67, 30:1.67, 31:1.67, 32:1.68, 33:1.68, 34:1.68, 35:1.68, 36:1.69, 37:1.69, 38:1.69, 
                       39:1.69, 40: 1.69, 41:1.70, 42:1.70, 43:1.70, 44:1.70, 45:1.70, 46:1.70, 47:1.70, 48:1.70, 49:1.71, 50:1.71}
    
    Random_Index = Random_Index_vals.get(n)
else:
    Random_Index = float(input("Enter the Random Index according to no. of criterias: "))


print("RI: ", Random_Index)
Consistency_Ratio = Consistency_index / Random_Index
print("CR: ", Consistency_Ratio)

CI:  0.0018934360424383012
RI:  1.61
CR:  0.0011760472313281374


In [6]:
#Importing Normalized data sheet and rearranging the columns.
Normalized_Data = pd.read_excel("Normalized Data Sheet.xlsx")
Normalized_Data = Normalized_Data.set_index("STRETCH NO.")
Normalized_Data_1 = Normalized_Data.groupby(Normalized_Data.columns.str[:2],axis=1, sort = False).sum()
print(Normalized_Data_1)
Normalized_Data_1.to_excel("Check Normalized Data.xlsx")
Normalized_Data_Matrix = np.array(Normalized_Data_1)

              A_   B_   L_   T_   PH   RA   R_   S_   E_   D_   P_   M_   PO  \
STRETCH NO.                                                                    
S1            73  115  117  157   41  178    0   20  157  172   45   36   26   
S2           151  203  122  175  200  130    0   73  160  190   58   50   52   
S3           230   77   80  111  158  115    0   20   92  107   45   95   69   
S4            78  168  125  180  216  168    0  140  185  160   70  123   69   
S5           190   91  103  116  200  153  255  176  275  242   95   95   45   
S6           183  115  154  151  158  145    0  246  146  175   25   85   65   
S7           151   78  140  167  175  109    0  250  207  217   91   55   68   
S8           113   91  148  133  158  128    0  123  196  197  250  134   68   
S9           131  111  137  112  200  116    0   53  142  172  108    0   65   
S10           53  227  122   94  225  190  100   20  157  210  183  191   71   
S11            0   34   59  121   66   3

In [7]:
#Ranking of Stretches - if values are consistent.
if Consistency_Ratio < 0.10:
    Consistency_Ratio = np.dot(Normalized_Data_Matrix, Criteria_Wts)
    AHP_Ranking = pd.DataFrame()
    AHP_Ranking["Priority Index"] = Consistency_Ratio
    AHP_Ranking["STRETCHES"] = Normalized_Data_1.index
    AHP_Ranking = AHP_Ranking.set_index("STRETCHES")
    AHP_Ranking["Rank"] = AHP_Ranking["Priority Index"].rank(ascending = False, method = 'max')
    print(AHP_Ranking)
    AHP_Ranking.to_excel("AHPRanking.xlsx")
else:
    print('The provided metrics are "Inconsistent".')

           Priority Index  Rank
STRETCHES                      
S1              74.865208  14.0
S2             102.021186   8.0
S3              79.344676  12.0
S4             110.516642   6.0
S5             130.334872   1.0
S6             109.887496   7.0
S7             117.433536   3.0
S8             116.454009   4.0
S9              95.469406   9.0
S10            115.683938   5.0
S11             54.821420  15.0
S12            129.450158   2.0
S13             88.452212  11.0
S14             76.843442  13.0
S15             93.225447  10.0
S16              9.456708  21.0
S17             10.453769  20.0
S18             21.116622  16.0
S19             13.648759  19.0
S20             15.460757  18.0
S21             15.539346  17.0
